# Comparison with article

In [1]:
from coupled_cluster.schemes.ccd_sparse import CoupledClusterDoublesSparse
from coupled_cluster.hartree_fock.scf_rhf import scf_rhf
from coupled_cluster.matrix_elements.generate_matrices import (
    get_one_body_elements, get_coulomb_elements,
    get_antisymmetrized_elements, add_spin_to_one_body_elements,
    get_one_body_elements_spin
)
from coupled_cluster.hartree_fock.basis_transformation import (
    transform_one_body_elements, transform_two_body_elements
)

import numpy as np
import time
import os

In [2]:
file_path = os.path.join("..", "dat")
filename = os.path.join(file_path, "coulomb_{0}.pkl")

omega = 1.0
l = 110
n = 12
theta = 0.8

filename = filename.format(l)

print ("""
w = {0},
l = {1},
n = {2},
theta = {3},
filename = {4}
""".format(omega, l, n, theta, filename))


w = 1.0,
l = 110,
n = 12,
theta = 0.8,
filename = ../dat/coulomb_110.pkl



In [6]:
h = omega * get_one_body_elements(l)
t0 = time.time()
u = np.sqrt(omega) * get_coulomb_elements(l, filename=filename, tol=1e-12)
t1 = time.time()
print ("Time spent creating Coulomb elements: {0} sec".format(t1 - t0)) 

t0 = time.time()
c, energy = scf_rhf(h.todense(), u, np.eye(l//2), n//2)
t1 = time.time()
print ("Time spent in SCF RHF: {0} sec".format(t1 - t0))

print ("\tRHF Energy: {0}".format(energy))

hi = transform_one_body_elements(h, c)
t0 = time.time()
oi = transform_two_body_elements(u, c)
t1 = time.time()
print ("Time spent transforming two body elements: {0} sec".format(t1 - t0))

_h = add_spin_to_one_body_elements(hi, l)
t0 = time.time()
_u = get_antisymmetrized_elements(l, oi=oi, tol=1e-12)
t1 = time.time()
print ("Time spent antisymmetrizing two body elements: {0} sec".format(t1 - t0))

t0 = time.time()
ccd_hf = CoupledClusterDoublesSparse(_h, _u, n)
t1 = time.time()
print ("Time spent setting up CCD code with HF basis: {0} sec".format(t1 - t0))
t0 = time.time()
energy, iterations = ccd_hf.compute_energy(tol=1e-4, theta=theta)
t1 = time.time()
print ("Time spent computing CCD energy with HF basis: {0} sec".format(t1 - t0))
print ("\tCCD (HF) Energy: {0}\n\tIterations: {1}\n\tSecond/iteration: {2}".format(energy, iterations, (t1 - t0)/iterations))

__h = omega * get_one_body_elements_spin(l)
t0 = time.time()
__u = np.sqrt(omega) * get_antisymmetrized_elements(l, filename=filename)
t1 = time.time()
print ("Time spent getting antisymmetric two body elements: {0} sec".format(t1 - t0))

t0 = time.time()
ccd = CoupledClusterDoublesSparse(__h, __u, n)
t1 = time.time()
print ("Time spent setting up CCD code with HO basis: {0} sec".format(t1 - t0))
t0 = time.time()
energy, iterations = ccd.compute_energy(theta=theta)
t1 = time.time()
print ("Time spent computing CCD energy with HO basis: {0} sec".format(t1 - t0))
print ("\tCCD Energy: {0}\n\tIterations: {1}\n\tSecond/iteration: {2}".format(energy, iterations, (t1 - t0)/iterations))

Time spent creating Coulomb elements: 0.014165401458740234 sec
Time spent in SCF RHF: 1.668471097946167 sec
	RHF Energy: 66.91203513015792
Time spent transforming two body elements: 1.6217572689056396 sec
Time spent antisymmetrizing two body elements: 3.9934065341949463 sec
Time spent setting up CCD code with HF basis: 1.9100799560546875 sec
Energy(0): 65.92007084659997
Energy(1): 65.92417143718545
Energy(2): 65.92525544511189
Energy(3): 65.9244515521034
Energy(4): 65.92252655453329
Energy(5): 65.9199951258632
Energy(6): 65.91719684823983
Energy(7): 65.91435033114868
Energy(8): 65.91159121430107
Energy(9): 65.90899880372052
Energy(10): 65.9066146803206
Energy(11): 65.90445563916563
Energy(12): 65.90252262918219
Energy(13): 65.9008068772314
Energy(14): 65.89929403621161
Energy(15): 65.89796695235957
Energy(16): 65.89680747302806
Energy(17): 65.8957975924947
Energy(18): 65.89492014535256
Energy(19): 65.89415919449509
Energy(20): 65.89350021633258
Energy(21): 65.8929301544545
Energy(22): 

KeyboardInterrupt: 

In [ ]:
print (h.density)
print (u.density)
print (_u.density)
print (_h.density)
print (__u.density)
print (__h.density)